# 🐶🐱 Classificação de Imagens: Gatos vs. Cachorros

Este notebook demonstra **como treinar um modelo simples de deep learning** para classificar imagens de **gatos e cachorros** usando TensorFlow e TensorFlow Datasets.

Compatível com **VS Code + Windows**.

In [1]:
# Instalar dependências (execute apenas uma vez no ambiente)
!pip install tensorflow tensorflow-datasets matplotlib

## 1. Importar bibliotecas principais

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)

TensorFlow: 2.20.0


## 2. Carregar o dataset `cats_vs_dogs`

O dataset vem inteiro como `train`. Precisamos **dividir manualmente em treino e teste**.

In [3]:
(dataset, ds_info) = tfds.load(
    'cats_vs_dogs',
    split='train',
    as_supervised=True,
    with_info=True
)

num_examples = ds_info.splits['train'].num_examples
train_size = int(0.8 * num_examples)

ds_train = dataset.take(train_size)
ds_test = dataset.skip(train_size)

print(f"Total de imagens: {num_examples}")
print(f"Treino: {train_size}, Teste: {num_examples - train_size}")

c:\QuartoSemestre\laboratorio-IV\reconhecimento_imagens\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 26.50 url/s]


KeyError: "There is no item named 'PetImages\\\\Cat\\\\0.jpg' in the archive"

## 3. Visualizar algumas imagens

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(dataset.take(9)):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title("Cachorro" if label.numpy() == 1 else "Gato")
    plt.axis("off")
plt.show()

## 4. Pré-processamento das imagens
- Redimensionar para 128x128
- Normalizar valores de pixel para `[0,1]`

In [ ]:
IMG_SIZE = 128

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

## 5. Criar o modelo de rede neural convolucional (CNN)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 6. Treinar o modelo

In [ ]:
history = model.fit(ds_train, validation_data=ds_test, epochs=5)

## 7. Avaliar o modelo

In [ ]:
loss, acc = model.evaluate(ds_test)
print(f"Acurácia no teste: {acc*100:.2f}%")